In [ ]:
!pip install "paho-mqtt<2.0.0"

# Successfully tested locally, no issues on the first run,
# but from the second run onwards, repeated multiple connections to the broker occur
import paho.mqtt.client as mqtt
import random
import time
import threading

broker_address = "broker.emqx.io"
port = 1883
username = "test"
password = "1"

# Global dictionary to track the connection status of each client
client_connections = {}

# Define the function to publish heartbeat data
def publish_heartbeat(client_name, topic):
    client_connected = threading.Event()

    def on_connect(client, userdata, flags, rc):
        if rc == 0:
            print(f"{client_name} connected to broker")
            client_connections[client_name] = True  # Update connection status
            client_connected.set()  # Set the flag indicating the connection is established
        else:
            print(f"{client_name} connection failed with code {rc}")

    def on_disconnect(client, userdata, rc):
        print(f"{client_name} disconnected from broker with code {rc}")
        client_connections[client_name] = False  # Update connection status
        client_connected.clear()  # Clear the flag indicating the connection is lost

    if client_connections.get(client_name):
        print(f"{client_name} already connected, skipping...")
        return

    client = mqtt.Client(client_id=f"{client_name}_{int(time.time())}")
    client.username_pw_set(username, password)
    client.on_connect = on_connect
    client.on_disconnect = on_disconnect

    client.connect(broker_address, port)
    client.loop_start()

    # Wait for the client to connect successfully
    if not client_connected.wait(timeout=10):
        print(f"{client_name} failed to connect within timeout")
        return

    while True:
        heartbeat = random.randint(60, 130)
        client.publish(topic, heartbeat)
        print(f"{client_name} sent heartbeat: {heartbeat}")
        time.sleep(5)

# Create multiple threads, each responsible for one client
threads = []

client_info = [
    {"client_name": "client1", "topic": "patient/num1/heartbeat"},
    {"client_name": "client2", "topic": "patient/num2/heartbeat"},
    {"client_name": "client3", "topic": "patient/num3/heartbeat"},
]

# Initialize connection status
for info in client_info:
    client_connections[info["client_name"]] = False

for info in client_info:
    thread = threading.Thread(target=publish_heartbeat, args=(info["client_name"], info["topic"]))
    threads.append(thread)
    thread.start()

# Keep the main thread running
try:
    for thread in threads:
        thread.join()
except KeyboardInterrupt:
    print("Stopping all clients...")
    for info in client_info:
        client = mqtt.Client(client_id=f"{info['client_name']}_{int(time.time())}")
        client.disconnect()
    print("All clients stopped.")
